# JupyterHub Notebook

### This notebook server is hosted on the OpenShift platform which provides a separate server for each individual user. The platform takes care of the provisioning of the server and allocating related to storage.

### First, install and import required libraries, watermark our file, initialise our Spark Session Builder and initialise our environment with required configuration

In [9]:
%pip install watermark
# %pip install Minio
# %pip install pyspark
# %pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


In [11]:
import os
import json
from pyspark import SparkConf
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.functions import from_json, col, to_json, struct
import watermark
from minio import Minio

%matplotlib inline

ModuleNotFoundError: No module named 'pyspark'

In [10]:
%load_ext watermark
%watermark -n -v -m -g -iv


Python implementation: CPython
Python version       : 3.8.6
IPython version      : 7.22.0

Compiler    : GCC 8.4.1 20200928 (Red Hat 8.4.1-1)
OS          : Linux
Release     : 4.18.0-193.41.1.el8_2.x86_64
Machine     : x86_64
Processor   : x86_64
CPU cores   : 16
Architecture: 64bit

Git hash: 93e9f5d64140a71ac414b9ac94fbebc43454ef5e

json: 2.0.9



In [6]:
if os.getenv('SPARK_CLUSTER') is None:
    with open('spark-info.txt', 'r') as sparkinfo:
        spark_cluster_name = sparkinfo.read()
    os.environ['SPARK_CLUSTER'] = spark_cluster_name

In [4]:
sparkSessionBuilder = SparkSession\
    .builder\
    .appName("Customer Churn ingest Pipeline")

In [5]:
os.environ['PYSPARK_SUBMIT_ARGS'] = \
'--packages \
org.postgresql:postgresql:42.2.10,\
org.apache.spark:spark-sql-kafka-0-10_2.11:2.4.5,\
org.apache.kafka:kafka-clients:2.4.0,\
org.apache.spark:spark-streaming_2.11:2.4.5,\
org.apache.hadoop:hadoop-aws:2.7.3 \
--conf spark.jars.ivy=/tmp \
--conf spark.hadoop.fs.s3a.endpoint=' + os.environ['S3_ENDPOINT'] + ' \
--conf spark.hadoop.fs.s3a.access.key=minio \
--conf spark.hadoop.fs.s3a.secret.key=minio123 \
--conf spark.hadoop.fs.s3a.path.style.access=true \
--conf spark.hadoop.fs.s3a.impl=org.apache.hadoop.fs.s3a.S3AFileSystem \
--master spark://' + os.environ['SPARK_CLUSTER'] + ':7077 pyspark-shell '


###  Connect to Spark Cluster provided by OpenShift Platform

In [6]:
spark = sparkSessionBuilder.getOrCreate()
spark.sparkContext.setLogLevel("INFO")
print('Spark context started.')

FileNotFoundError: [Errno 2] No such file or directory: '/opt/app-root/lib/python3.6/site-packages/pyspark/./bin/spark-submit'

###  Declare our input data sources, import and combine them

In [ ]:
dataFrame_Customer = spark.read\
                .options(delimeter=',', inferSchema='True', header='True') \
                .csv("s3a://rawdata/Customer-Churn_P1.csv")
dataFrame_Customer.printSchema()

In [ ]:
# dataFrame_Products = spark.read\
#                 .options(delimeter=',', inferSchema='True', header='True') \
#                 .csv("s3a://rawdata/Customer-Churn_P2.csv")
# dataFrame_Products.printSchema()


In [ ]:
from pyspark.sql.types import *
from  pyspark.sql.functions import *

srcKafkaBrokers = "odh-message-bus-kafka-bootstrap:9092"
srcKakaTopic = "datatelco"



schema = StructType()\
    .add("customerID", IntegerType())\
    .add("PhoneService", StringType())\
    .add("MultipleLines", StringType())\
    .add("InternetService", StringType())\
    .add("OnlineSecurity", StringType())\
    .add("OnlineBackup", StringType())\
    .add("DeviceProtection", StringType())\
    .add("TechSupport", StringType())\
    .add("StreamingTV", StringType())\
    .add("StreamingMovies", StringType())\
    .add("Contract", StringType())\
    .add("PaperlessBilling", StringType())\
    .add("PaymentMethod", StringType())\
    .add("MonthlyCharges", StringType())\
    .add("TotalCharges", DoubleType())\
    .add("Churn", StringType())



#Read from JSON Kafka messages into a dataframe
dfKafka = spark.read.format("kafka")\
    .option("kafka.bootstrap.servers", srcKafkaBrokers)\
    .option("subscribe", srcKakaTopic)\
    .option("startingOffsets", "earliest")\
    .load()\
    .withColumn("value", regexp_replace(col("value").cast("string"), "\\\\", "")) \
    .withColumn("value", regexp_replace(col("value"), "^\"|\"$", "")) \
    .selectExpr("CAST(value AS STRING) as jsonValue")\
    .rdd.map(lambda row: row["jsonValue"])

dfObj = spark.read.schema(schema).json(dfKafka)
dfObj.printSchema()
dfObj.show(n=2)


In [ ]:
dataFrom_All = dataFrame_Customer.join(dfObj, "customerID", how="full")

###  Push prepared data to object storage and stop Spark cluster to save resources
###  Note - be sure to change this user_id on the next line to your username (something in the range user1 ... user30)

In [ ]:
user_id = "user29"
file_location = "s3a://data/full_data_csv" + user_id
dataFrom_All.repartition(1).write.mode("overwrite")\
    .option("header", "true")\
    .format("csv").save(file_location)

In [ ]:
spark.stop()